In [2]:
#!pip install pandas

In [340]:
import json
import pandas as pd
from collections import defaultdict
from itertools import combinations,permutations 
import numpy as np
from sklearn import cluster
import glob
import pickle
import warnings
warnings.filterwarnings("ignore")

#### Data Processing

In [157]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'split_merged_dataset/*_train.jsonl')
all_datasets
all_datasets[3].split('/')[-1][:-6].split('_')[0]

'ta'

In [158]:
languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    languages.append(lang)
print(languages)

['pa', 'gu', 'hi', 'ta', 'as', 'ml', 'te', 'kn', 'bn', 'or', 'mr', 'en']


In [159]:
ent_dict = defaultdict()
fact_dict_all = defaultdict()

languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    
    e_dict = {}
    print(lang)
    with open(dataset) as f:
        
        
        unique_facts = []

        for line in f:
    #         if i>50:
    #             break
            #print(i)
            data = json.loads(line)
            f = []
            for l in data['facts_list']:
                #print('sentence')
                f2 = []
                for fact in l:
                    f2.append(fact[0])  
                    # Generate facts 
                    unique_facts.append(fact[0])  
                f.append(f2)
                #print(f)
            e_dict[data['entity_name']] = f
            i+=1
    languages.append(lang)
    fact_dict_all[lang] = set(unique_facts)
    print(lang,len(fact_dict_all[lang]))
    ent_dict[lang] = e_dict

pa
pa 218
gu
gu 133
hi
hi 216
ta
ta 225
as
as 136
ml
ml 244
te
te 166
kn
kn 165
bn
bn 245
or
or 141
mr
mr 189
en
en 301


#### Affinity Matrix

In [505]:
#print(languages)

In [566]:
fact_dict = defaultdict()
factsetdict = defaultdict()

for lang in languages:
    print(lang)
    fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
    factsetdict[lang] = set(fact_dict_all[lang])
    
    fact_dict[lang] = fdict
    print(fact_dict) 

pa
defaultdict(None, {'pa': {0: 'affiliation', 1: 'coach of sports team', 2: "topic's main template", 3: 'conflict', 4: 'successful candidate', 5: 'Elo rating', 6: 'replaced by', 7: 'significant person', 8: 'military rank', 9: 'time period', 10: 'participant in', 11: 'defendant', 12: 'statistical leader', 13: 'architect', 14: 'publication date', 15: 'charge', 16: 'filmography', 17: 'name in native language', 18: 'residence', 19: 'designed by', 20: 'member of', 21: 'place of burial', 22: 'choreographer', 23: 'winner', 24: 'producer', 25: 'sports discipline competed in', 26: 'editor-in-chief', 27: 'doctoral advisor', 28: 'candidacy in election', 29: 'has written for', 30: 'position played on team / speciality', 31: 'operator', 32: 'composer', 33: 'contributed to creative work', 34: 'archives at', 35: 'flag bearer', 36: 'mass', 37: 'performer', 38: 'part of', 39: 'number of children', 40: 'member of the crew of', 41: 'film editor', 42: 'work period (end)', 43: 'secretary general', 44: 'mo

In [567]:
perm_dict = defaultdict()
for i,(k,facts) in enumerate(factsetdict.items()):
    perm_dict[k] = list(permutations(facts,2))

#### PMI 

In [14]:
# data=pd.read_csv('../datasets/fact_pair_probs/en.tsv',sep='\t',header=None)
# data.columns = ['relation','scores']
# data.head(5)

In [568]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'fact_pair_probs/*.tsv')
all_datasets 

['/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ta.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/bn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/mr.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/te.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/as.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/gu.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/hi.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/en.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/pa.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/kn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/or.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ml.tsv']

In [569]:
pmi_dict = defaultdict()
for lang in languages:
    for dataset in all_datasets:
        if lang == dataset.split('/')[-1][:-6].split('_')[0]:
            pmi_list = []
            with open(dataset) as f:
                for line in f:
                    l = line.split('\t')
                    pmi_list.append(l)
    pmi_dict[lang] = pmi_list

In [523]:
#pmi_dict

#### Affinity Matrix

In [40]:
affinity_dict = defaultdict()
for lang in languages: 
    print(lang)
    a_dict = defaultdict()
    for p in perm_dict[lang]:
        #print(p)
        look = p[0] + '|' +  p[1]
        look2 = p[1] + '|' +  p[0]
        for item in pmi_dict[lang]:
            if look == item[0] or look2 == item[0]:
                # print("yes")
                #print(p)
                a_dict[p] = float(item[1].split('\n')[:-1][0])
                break
            else:
                a_dict[p] = float(0)
    affinity_dict[lang] = a_dict

te


NameError: name 'perm_dict' is not defined

#### Create affinity matrix

In [572]:
for lang in languages:
    print(lang)
    if lang=='en':
    s = len(fact_dict[lang]),len(fact_dict[lang])
    affinity_matrix  = np.zeros(s)
  
    for r in fact_dict[lang].keys():
        print(r)
#        print(r)
        cols = np.zeros(len(fact_dict[lang].keys()))
        for c in fact_dict[lang].keys():
            for item in affinity_dict[lang].keys():
               # print(item)
                #print(r,item[0])
                if fact_dict[lang][r] == item[0] and fact_dict[lang][c] == item[1]:
                    #print("yes")
                    affinity_matrix[r][c] = affinity_dict[lang][item]
                    affinity_matrix[c][r] = affinity_dict[lang][item]

    np.save(f'../datasets/affinity_matrices/{lang}_affinity_matrix.npy',affinity_matrix)   


pa
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
gu
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80

KeyError: 'en'

##### Spectral clustering for all entities on validation set

In [290]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/"
all_datasets = glob.glob(root  + '*_test.jsonl')
all_datasets 

['/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/te_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/gu_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/as_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/or_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ta_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/bn_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/hi_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/pa_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/en_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/mr_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ml_merged_test.jsonl',

In [51]:
#val_dataset = '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/' + lang + '_merged_test.jsonl

In [291]:
ent_dict = defaultdict()
fact_dict_all = defaultdict()
cnts_dict = {}
languages = []

languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    print(lang)
    
    e_dict = {}
    
    cnts = []
    
    with open(dataset) as f:
        
        
        unique_facts = []

        for line in f:
            c = 0
            data = json.loads(line)
            f = []
            for l in data['facts_list']:
                #print('sentence')
                f2 = []
                for fact in l:
                    f2.append(fact[0])  
                    # Generate facts 
                    unique_facts.append(fact[0])  
                f.append(f2)
                #print(f)
                c+=1
            
            cnts.append(c)
    
            e_dict[data['entity_name']] = f
        
    languages.append(lang)
    fact_dict_all[lang] = set(unique_facts)
    print(lang,len(fact_dict_all[lang]))
    ent_dict[lang] = e_dict
    cnts_dict[lang] = cnts

te
te 105
gu
gu 81
as
as 77
or
or 80
ta
ta 148
bn
bn 170
hi
hi 145
pa
pa 151
en
en 225
mr
mr 120
ml
ml 164
kn
kn 105


In [292]:
cnts_dict_ent = defaultdict()
for i,lang in enumerate(cnts_dict):
    num_sent = pd.Series(cnts_dict[lang]).sum()
    #print(lang,num_sent)
    cnts_dict_ent[lang]=num_sent
print(cnts_dict_ent)

defaultdict(None, {'te': 2459, 'gu': 706, 'as': 818, 'or': 1691, 'ta': 6548, 'bn': 13285, 'hi': 5750, 'pa': 4024, 'en': 18003, 'mr': 2506, 'ml': 6343, 'kn': 1760})


In [202]:
pd.Series(cnts_dict['te']).value_counts()

2     134
1     127
3     109
4      84
5      65
6      48
7      37
8      26
9      19
10     15
dtype: int64

##### Clustering on a dataset

In [375]:
fact_dict = defaultdict()
factsetdict = defaultdict()

for lang in languages:
    print(lang)
    fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
    factsetdict[lang] = set(fact_dict_all[lang])
    
    fact_dict[lang] = fdict
    #print(fact_dict) 

te
gu
as
or
ta
bn
hi
pa
en
mr
ml
kn


In [374]:
#fact_dict['te'][]

In [373]:
#fact_dict.values()

In [359]:
#ent_dict 
# for e,c in zip(ent_dict.keys(),cnts):
#     print(e,c)
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

entpred_dict_all = defaultdict()

cnt =0
nothing=0
for lang in languages:
    #if lang == 'te':
    print(lang)
#         cnt+=1
#         if cnt>1:
#             break
    affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
    print("loaded affinity Matrix")

    entpred_dict  = defaultdict()

    for i,v in enumerate(ent_dict[lang].keys()):
        #print(v)
        #print(v,ent_dict[v])
        e = ent_dict[lang][v]
        c = cnts_dict[lang][i]
        #print(v,e,c)
        #print(c)

        flat_list = [item for sublist in e for item in sublist]
        flat_set = set(flat_list)
        #print(flat_set)
        flat_ids = []

        for item in flat_set:
            for i,(key,value) in enumerate(fact_dict[lang].items()):
                if value==item:
                    #print(value)
                    flat_ids.append(key)
        #print(flat_ids)
        ent_perm_list = list(permutations(flat_ids,2))

        # Special case - for all occupation occurences in multiple sentences
        oc=-1
        for sublist in e: 
            #print(sublist)
            if sublist.count(sublist[0]) == len(sublist):
                for item in sublist:
                    #print(item)
                    if item == 'occupation':
                        oc+=1 
                        break


        ocl = [list(fact_dict[lang].keys())[list(fact_dict[lang].values()).index('occupation')] for i in range(oc)]
        #print(ocl)
        flat_ids.extend(ocl)
        #print(flat_ids)

        ## Special case ends

        ent_aff_matrix = np.zeros((len(flat_ids),len(flat_ids)))
        #print(ent_aff_matrix.shape)


        for p in ent_perm_list:
            ent_aff_matrix[flat_ids.index(p[0]),flat_ids.index(p[1])] = affinity_matrix[p[0],p[1]]

        #print(ent_aff_matrix)


        # clusters
        try:
            ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)
            #print("Clusters",ent_clusters)
        except Exception:
            #nothing +=1
            #print(nothing)
            #print("No clusters")
            #print(ent_aff_matrix.shape,c)
            continue

        # Group facts by clusters
        d = defaultdict()
        for i,e in zip(flat_set,ent_clusters):
            d[i]=e
        #print(d)

        x = defaultdict(list)

        for key, value in sorted(d.items()):
            x[value].append(key)

        pred_list = list(x.values())
        #print(pred_list)
        #print(v)

        entpred_dict[v] = pred_list


    entpred_dict_all[lang] = entpred_dict

te
loaded affinity Matrix
gu
loaded affinity Matrix
as
loaded affinity Matrix
or
loaded affinity Matrix
ta
loaded affinity Matrix
bn
loaded affinity Matrix
hi
loaded affinity Matrix
pa
loaded affinity Matrix
en
loaded affinity Matrix
mr
loaded affinity Matrix
ml
loaded affinity Matrix
kn
loaded affinity Matrix


##### Store validation cluster predictions

In [207]:
#entpred_dict_all['te']

In [360]:
entpred_dict_all.keys()

dict_keys(['te', 'gu', 'as', 'or', 'ta', 'bn', 'hi', 'pa', 'en', 'mr', 'ml', 'kn'])

In [361]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/cluster_predictions/"
with open(root + 'cluster_predictions_all.pkl', 'wb') as f:
    pickle.dump(entpred_dict_all, f)

##### Accuracy on validation set

In [362]:
len(entpred_dict_all['te'])

536

In [363]:
scores_dict = defaultdict()
#l = ['te']
for i,lang in enumerate(entpred_dict_all):
    #print(i)
    print(lang)
    
    total_sent = 0
    total_right = 0 
    total_ones = 0
    total_gtpart = 0
    total_gt1sent = 0
    acc_dict = {}
    ent_sent_correct = 0
    
    scores = defaultdict()
    
    for j,(k,v) in enumerate(entpred_dict_all[lang].items()):
        #print(i)
        #print(k)
        act = ent_dict[lang][k]

    #     print("Pred",k,v)
    #     print("Act",act)

        
        num_sent = len(v)
        total_sent += num_sent
        #print(num_sent)

        pred = (v)
        right=0;ones=0
        gtpart_right=0

        # only one sentence and fact
        if num_sent==1: #and len(v[0])==1:
            ones+=1
            #right+=1
            #print(pred)
            
        else :
            total_gt1sent +=num_sent
            for sent in pred:
                for act_sent in act:
                    if sorted(sent) == sorted(act_sent):
                        #print(sorted(sent),sorted(act_sent))
                        right +=1
                        #print(right)
                        break


        #acc = right/num_sent
        total_right += right


        # Check partial or fully matched sentences 
        d = np.zeros(len(act))


        for sent in pred:
            
            for i,act_sent in enumerate(act):
                if d[i] == 1:
                    continue


                if(set(sent).issubset(set(act_sent))):
                    d[i] = 1
                    #print("partial match",sorted(sent),sorted(act_sent))

                    gtpart_right +=1
                    break

        total_gtpart += gtpart_right                
        #print(gtpart_right,total_gtpart) 

        total_ones += ones

        
        if right == num_sent:
            ent_sent_correct +=1
            
        
    #print(i)       
    scores['total_sent'] = total_sent
    scores['total_right'] = total_right
    scores['total_ones'] = total_ones
    scores['sent_gt1'] = total_gt1sent
    scores['total_gtpart'] = total_gtpart
    scores['ent_sent_correct'] = ent_sent_correct
    scores['ent'] = j+1
    
    print(total_sent)  
    print(total_right)
    print(total_ones)
    print(total_gtpart)
    
    scores['sub_acc']  = round(total_gtpart/total_sent,2)
    scores['acc'] = round(total_right/total_gt1sent,2)
    scores['ent_pc_correct'] = round(ent_sent_correct/(j+1),2)
    scores_dict[lang] = scores

te
1496
529
140
1169
gu
517
169
59
405
as
399
158
34
324
or
766
304
65
656
ta
3297
753
845
2143
bn
6876
1563
609
4464
hi
3536
702
869
2390
pa
2823
614
362
1939
en
12474
1662
3217
7684
mr
2096
362
730
1657
ml
3560
844
508
2297
kn
1038
333
106
775


In [366]:
scores_dict['te']

defaultdict(None,
            {'total_sent': 1496,
             'total_right': 529,
             'total_ones': 140,
             'sent_gt1': 1356,
             'total_gtpart': 1169,
             'ent_sent_correct': 64,
             'ent': 536,
             'sub_acc': 0.78,
             'acc': 0.39,
             'ent_pc_correct': 0.12})

In [370]:
cols  =['language','total_ent','total_sent_all','total_sent','total_sent_cov','sent_gt1','total_right','accuracy_abs_match','acc_sub_match','ent','ent_sent_correct','ent_pc_correct']
df = pd.DataFrame(columns=cols)

for i,lang in enumerate(scores_dict):
    #print(lang)
    df2 = [{'language': lang, 'total_ent':len(ent_dict[lang]),
            'total_sent_all':cnts_dict_ent[lang],
            'total_sent':scores_dict[lang]['total_sent'],
            'total_sent_cov':round(scores_dict[lang]['total_sent']/cnts_dict_ent[lang],2),
       'sent_gt1':scores_dict[lang]['sent_gt1'],
        'total_right':scores_dict[lang]['total_right'],
        'accuracy_abs_match':scores_dict[lang]['acc']
       ,'acc_sub_match':scores_dict[lang]['sub_acc']
        ,'ent':scores_dict[lang]['ent']
        ,'ent_sent_correct':scores_dict[lang]['ent_sent_correct']
        ,'ent_pc_correct':scores_dict[lang]['ent_pc_correct']}]
    #print(df2)
    df2= pd.DataFrame.from_dict(df2,orient='columns')
    #print(df2)
    
    df = pd.concat([df,df2], ignore_index=True)
    #df.append(df2)
    
print(df)       

   language total_ent total_sent_all total_sent  total_sent_cov sent_gt1  \
0        te       664           2459       1496            0.61     1356   
1        gu       212            706        517            0.73      458   
2        as       189            818        399            0.49      365   
3        or       411           1691        766            0.45      701   
4        ta      2356           6548       3297            0.50     2452   
5        bn      3492          13285       6876            0.52     6267   
6        hi      2178           5750       3536            0.61     2667   
7        pa      1282           4024       2823            0.70     2461   
8        en      7554          18003      12474            0.69     9257   
9        mr      1270           2506       2096            0.84     1366   
10       ml      1966           6343       3560            0.56     3052   
11       kn       476           1760       1038            0.59      932   

   total_ri

In [371]:
df

,language,total_ent,total_sent_all,total_sent,total_sent_cov,sent_gt1,total_right,accuracy_abs_match,acc_sub_match,ent,ent_sent_correct,ent_pc_correct
0,te,664,2459,1496,0.61,1356,529,0.39,0.78,536,64,0.12
1,gu,212,706,517,0.73,458,169,0.37,0.78,191,16,0.08
2,as,189,818,399,0.49,365,158,0.43,0.81,133,17,0.13
3,or,411,1691,766,0.45,701,304,0.43,0.86,285,33,0.12
4,ta,2356,6548,3297,0.50,2452,753,0.31,0.65,1711,103,0.06
5,bn,3492,13285,6876,0.52,6267,1563,0.25,0.65,2475,84,0.03
6,hi,2178,5750,3536,0.61,2667,702,0.26,0.68,1784,82,0.05
7,pa,1282,4024,2823,0.70,2461,614,0.25,0.69,1084,58,0.05
8,en,7554,18003,12474,0.69,9257,1662,0.18,0.62,6418,210,0.03
9,mr,1270,2506,2096,0.84,1366,362,0.27,0.79,1206,42,0.03


In [298]:
### Without occupation handling
df

,language,total_ent,total_sent_all,total_sent,sent_gt1,total_right,accuracy_abs_match,acc_sub_match,ent,ent_sent_correct,ent_pc_correct
0,te,664,2459,1288,1161,408,0.35,0.76,464,45,0.10
1,gu,212,706,467,410,143,0.35,0.77,172,13,0.08
2,as,189,818,302,273,117,0.43,0.79,104,12,0.12
3,or,411,1691,573,517,197,0.38,0.83,212,20,0.09
4,ta,2356,6548,3214,2391,737,0.31,0.65,1670,104,0.06
5,bn,3492,13285,6531,5941,1449,0.24,0.64,2359,82,0.03
6,hi,2178,5750,3382,2536,633,0.25,0.67,1725,71,0.04
7,pa,1282,4024,2643,2293,539,0.24,0.68,1022,50,0.05
8,en,7554,18003,12460,9248,1679,0.18,0.62,6405,203,0.03
9,mr,1270,2506,2025,1300,335,0.26,0.78,1179,43,0.04


In [372]:
df.to_csv("../results/clustering-scores-alllang-occ.csv",index=False)